Simple muon and neutrino flux calculation
-----------------------------------------

This notebook demonstrates the basic use case of the solver. It calculates the atmospheric lepton flux, taking most of the configuration values from the 'global' config file `mceq_config.py`. The interaction model and the primary cosmic ray flux model are selected during initialization.

In [1]:
#basic imports and ipython setup
%load_ext autoreload
%matplotlib inline
%autoreload 2
import os
import matplotlib.pyplot as plt
import numpy as np
os.chdir('..')


#import solver related modules
from MCEq.core import MCEqRun
from mceq_config import config
#import primary model choices
import CRFluxModels.CRFluxModels as pm

#import sys
#sys.path.append('/Users/mhuber/AtmosphericMuons_Anatoli/strawman-hadint/references')
#sys.path.append('/Users/mhuber/MatrixCascadeEquationCode/MCEq')

#import references as ref


Create an instance of an MCEqRun class. Most of its areguments are contained in the `config` dictionary from the `mceq_config` module. Look into or edit `mceq_config.py`.

If the initialization succeeds it will print out some information according to the debug level. 

In [2]:
print('###################################################')
print('Run the scripts with EPOS as hadronic interaction model')
print('###################################################')
'''
mceq_run_epos = MCEqRun(
#provide the string of the interaction model
interaction_model='SIBYLL2.3',
#primary cosmic ray flux model
#support a tuple (primary model class (not instance!), arguments)
primary_model=(pm.CombinedGHandHG, "H3a"),
# Zenith angle in degrees. 0=vertical, 90=horizontal
theta_deg=16.0,
#na49=True,
#expand the rest of the options from mceq_config.py
**config
)
'''
print('###################################################')
print('Run the scripts with Sibyll as hadronic interaction model')
print('###################################################')
mceq_run = MCEqRun(
#provide the string of the interaction model
interaction_model='SIBYLL2.3',
#primary cosmic ray flux model
#support a tuple (primary model class (not instance!), arguments)
primary_model=(pm.CombinedGHandHG, "H3a"),
# Zenith angle in degrees. 0=vertical, 90=horizontal
theta_deg=16.0,
na49_model=(None,'all'),
#expand the rest of the options from mceq_config.py
**config
)
#mceq_run.set_single_primary_particle(E=1.e5,corsika_id=201)

print('###################################################')
print('Now rerun everything with na49 data driven approach')
print('###################################################')
mceq_run_na49 = MCEqRun(
#provide the string of the interaction model
interaction_model='SIBYLL2.3',#'QGSJET-II-04',#
#primary cosmic ray flux model
#support a tuple (primary model class (not instance!), arguments)
primary_model=(pm.CombinedGHandHG, "H3a"),
# Zenith angle in degrees. 0=vertical, 90=horizontal
theta_deg=16.0,
#use na49 data
na49_model = ('pC',[211,-211]),
#expand the rest of the options from mceq_config.py
**config
)


###################################################
Run the scripts with EPOS as hadronic interaction model
###################################################
###################################################
Run the scripts with Sibyll as hadronic interaction model
###################################################
InteractionYields::_load(): Looking for /Users/mhuber/AtmosphericMuons_Anatoli/MCEq/data/SIBYLL23_yields_compact_ledpm.bz2
DecayYields:_load():: Loading file /Users/mhuber/AtmosphericMuons_Anatoli/MCEq/data/decays_v1_compact.ppd

Hadrons and stable particles:

"p", "p-bar", "n-bar", "n"

Mixed:

"pi-", "pi+", "K0L", "K-", "K+", "Lambda0", "Lambda0-bar", "K0S", 
"D+", "D-", "Ds+", "Ds-", "D0", "D0-bar"

Resonances:



Leptons:

"e-", "nue", "numu", "nutau", "antinutau", "antinumu", "antinue", "e+", 
"mu-", "mu+"

Aliases:
"obs_numu", "obs_nutau", "gamma", "pr_antinutau", "pr_antinumu", "pr_antinue", "obs_antinue", "k_nue", 
"k_numu", "k_nutau", "pi_antinutau", "pi_antinu

__________

If everything succeeds than the last message should be something like

`MCEqRun::set_primary_model():  HillasGaisser2012 H3a`.

The spline interpolating the depth-density relation of an atmosphere will be caluclated and cached during the first run. 

Integrate
---------
Now, run the solve command to integrate the cascade equations starting from the top of the atmosphere
down to the observation level `h_obs` (see config).

In [ ]:
#mceq_run.print_particle_tables()
print(mceq_run.max_ldec)
print(mceq_run._calculate_integration_path(None,'X'))
int_path = mceq_run.integration_path
print(len(int_path[1]))

In [ ]:
mceq_run.solve(int_grid=int_path[1])
print('Done')
mceq_run_na49.solve()
print('Done')


In [ ]:
print([p.name for p in mceq_run.particle_species])
print(mceq_run.dim_states/88)
print(mceq_run.alias_table)

In [ ]:
print(mceq_run.grid_sol)

Retrieve results
----------------

In [ ]:
print(len(mceq_run.grid_sol))
sol=mceq_run.get_solution('conv_mu+', 3,grid_idx=-1)
print(sol)

In [ ]:
###############################################
# Model results

#Power of energy to scale the flux
mag = 3

#obtain energy grid (nver changes) of the solution for the x-axis of the plots
e_grid = mceq_run.e_grid

#Retrieve the flux at the surface by using the aliases which were listed in the
#output during the MCEqRun initialization. 
flux = {}
#_conv means conventional (mostly pions and kaons)
flux['mu_conv'] = (mceq_run.get_solution('conv_mu+', mag)
                   + mceq_run.get_solution('conv_mu-', mag))

# _pr means prompt (the mother of the muon had a critical energy
# higher than a D meson. Includes all charm and direct resonance
# contribution)
flux['mu_pr'] = (mceq_run.get_solution('pr_mu+', mag)
                 + mceq_run.get_solution('pr_mu-', mag))

# total means conventional + prompt
flux['mu_total'] = (mceq_run.get_solution('total_mu+', mag)
                    + mceq_run.get_solution('total_mu-', mag))

# same meaning of prefixes for muon neutrinos as for muons
flux['numu_conv'] = (mceq_run.get_solution('conv_numu', mag)
                     + mceq_run.get_solution('conv_antinumu', mag))

flux['numu_pr'] = (mceq_run.get_solution('pr_numu', mag)
                   + mceq_run.get_solution('pr_antinumu', mag))

flux['numu_total'] = (mceq_run.get_solution('total_numu', mag)
                      + mceq_run.get_solution('total_antinumu', mag))

# same meaning of prefixes for electron neutrinos as for muons
flux['nue_conv'] = (mceq_run.get_solution('conv_nue', mag)
                    + mceq_run.get_solution('conv_antinue', mag))

flux['nue_pr'] = (mceq_run.get_solution('pr_nue', mag)
                  + mceq_run.get_solution('pr_antinue', mag))

flux['nue_total'] = (mceq_run.get_solution('total_nue', mag)
                     + mceq_run.get_solution('total_antinue', mag))


# since there are no conventional tau neutrinos, prompt=total
flux['nutau_pr'] = (mceq_run.get_solution('total_nutau', mag)
                    + mceq_run.get_solution('total_antinutau', mag))



###############################################
# na49 results


#Retrieve the flux at the surface by using the aliases which were listed in the
#output during the MCEqRun initialization. 
flux_na49 = {}
#_conv means conventional (mostly pions and kaons)
flux_na49['mu_conv'] = (mceq_run_na49.get_solution('conv_mu+', mag)
                   + mceq_run_na49.get_solution('conv_mu-', mag))

# _pr means prompt (the mother of the muon had a critical energy
# higher than a D meson. Includes all charm and direct resonance
# contribution)
flux_na49['mu_pr'] = (mceq_run_na49.get_solution('pr_mu+', mag)
                 + mceq_run_na49.get_solution('pr_mu-', mag))

# total means conventional + prompt
flux_na49['mu_total'] = (mceq_run_na49.get_solution('total_mu+', mag)
                    + mceq_run_na49.get_solution('total_mu-', mag))

# same meaning of prefixes for muon neutrinos as for muons
flux_na49['numu_conv'] = (mceq_run_na49.get_solution('conv_numu', mag)
                     + mceq_run_na49.get_solution('conv_antinumu', mag))

flux_na49['numu_pr'] = (mceq_run_na49.get_solution('pr_numu', mag)
                   + mceq_run_na49.get_solution('pr_antinumu', mag))

flux_na49['numu_total'] = (mceq_run_na49.get_solution('total_numu', mag)
                      + mceq_run_na49.get_solution('total_antinumu', mag))

# same meaning of prefixes for electron neutrinos as for muons
flux_na49['nue_conv'] = (mceq_run_na49.get_solution('conv_nue', mag)
                    + mceq_run_na49.get_solution('conv_antinue', mag))

flux_na49['nue_pr'] = (mceq_run_na49.get_solution('pr_nue', mag)
                  + mceq_run_na49.get_solution('pr_antinue', mag))

flux_na49['nue_total'] = (mceq_run_na49.get_solution('total_nue', mag)
                     + mceq_run_na49.get_solution('total_antinue', mag))


# since there are no conventional tau neutrinos, prompt=total
flux_na49['nutau_pr'] = (mceq_run_na49.get_solution('total_nutau', mag)
                    + mceq_run_na49.get_solution('total_antinutau', mag))
'''
###############################################
# EPOS results


#Retrieve the flux at the surface by using the aliases which were listed in the
#output during the MCEqRun initialization. 
flux_epos = {}
#_conv means conventional (mostly pions and kaons)
flux_epos['mu_conv'] = (mceq_run_epos.get_solution('conv_mu+', mag)
                   + mceq_run_epos.get_solution('conv_mu-', mag))

# _pr means prompt (the mother of the muon had a critical energy
# higher than a D meson. Includes all charm and direct resonance
# contribution)
flux_epos['mu_pr'] = (mceq_run_epos.get_solution('pr_mu+', mag)
                 + mceq_run_epos.get_solution('pr_mu-', mag))

# total means conventional + prompt
flux_epos['mu_total'] = (mceq_run_epos.get_solution('total_mu+', mag)
                    + mceq_run_epos.get_solution('total_mu-', mag))

# same meaning of prefixes for muon neutrinos as for muons
flux_epos['numu_conv'] = (mceq_run_epos.get_solution('conv_numu', mag)
                     + mceq_run_epos.get_solution('conv_antinumu', mag))

flux_epos['numu_pr'] = (mceq_run_epos.get_solution('pr_numu', mag)
                   + mceq_run_epos.get_solution('pr_antinumu', mag))

flux_epos['numu_total'] = (mceq_run_epos.get_solution('total_numu', mag)
                      + mceq_run_epos.get_solution('total_antinumu', mag))

# same meaning of prefixes for electron neutrinos as for muons
flux_epos['nue_conv'] = (mceq_run_epos.get_solution('conv_nue', mag)
                    + mceq_run_epos.get_solution('conv_antinue', mag))

flux_epos['nue_pr'] = (mceq_run_epos.get_solution('pr_nue', mag)
                  + mceq_run_epos.get_solution('pr_antinue', mag))

flux_epos['nue_total'] = (mceq_run_epos.get_solution('total_nue', mag)
                     + mceq_run_epos.get_solution('total_antinue', mag))


# since there are no conventional tau neutrinos, prompt=total
flux_epos['nutau_pr'] = (mceq_run_epos.get_solution('total_nutau', mag)
                    + mceq_run_epos.get_solution('total_antinutau', mag))

'''

Plot with matplotlib
--------------------

In [ ]:
e_grid = mceq_run.y.e_grid
#get path of the home directory + Desktop
save_pdf = False
desktop = os.path.join(os.path.expanduser("~"),'Desktop')
for pref, lab in [('mu_',r'\mu'), ('numu_',r'\nu_\mu'), ('nue_',r'\nu_e')]:
    kw = dict(height_ratios=[3,1],hspace=0.2)
    fig, axes = plt.subplots(2,1,sharex=True,figsize=(7.,5.), gridspec_kw=kw,squeeze=False)
    for key,run in [('normal',flux), ('NA49',flux_na49)]:
        p0 = axes[0,0].semilogx(e_grid, run[pref + 'total'], ls='--', lw=2.,
                          label=r'total ${0}$, Model=${1}$'.format(lab,key))
        c0 = p0[0].get_color()
        axes[0,0].semilogx(e_grid, run[pref + 'conv'], color=c0, ls='-.', lw=1.5,
                     label=r'conventional ${0}$'.format(lab),zorder=2)
        axes[0,0].semilogx(e_grid, run[pref + 'pr'], color=c0,ls='--', lw=1.5, 
               label='prompt ${0}$'.format(lab),zorder=2)
    
        if key is not 'NA49':
            axes[1,0].plot(e_grid, run[pref + 'total']/flux_na49[pref+'total'], color=c0,ls='--', lw=2.5, 
               label='Total ${0}$'.format(lab))
    
    #axes[0,0].set_xlim(1,1e5)
    #axes[1,0].set_ylim(0.5,1.5)
    axes[1,0].set_xlabel(r"$E_{{{0}}}$ [GeV]".format(lab))
    axes[1,0].set_ylabel(r"Ratio (Flux/NA49)")
    axes[0,0].set_ylabel(r"$\Phi_{" + lab + "}$ (E/GeV)$^{" + str(mag) +" }$" + 
               "(cm$^{2}$ s sr GeV)$^{-1}$")
    axes[0,0].set_xlim(1,1e5)
    #axes[0,0].set_ylim(0,0.45)
    axes[0,0].legend(loc='upper right')
    #fig.tight_layout()
    
    if save_pdf: fig.savefig(os.path.join(desktop, pref + 'flux.pdf'))

pref = 'nutau'
if np.any(flux[pref + '_pr']): #for models without prompt -> no nutau
    #plt.figure(figsize=(7., 5.5))
    kw = dict()
    fig, axes = plt.subplots(1,1,sharex=True,figsize=(7.,7.), gridspec_kw=kw,squeeze=False)
    lab = r'\nu_\tau'
    axes[0,0].semilogx(e_grid, flux[pref + '_pr'], color='k', ls='-', lw=1.5,
               label='prompt ${0}$'.format(lab))
    
    #axes[0,0].set_xlim(1,1e9)
    #axes[0,0].set_ylim(1e-8,1e-3)
    axes[0,0].set_xlabel(r"$E_{{{0}}}$ [GeV]".format(lab))
    axes[0,0].set_ylabel(r"$\Phi_{" + lab + "}$ (E/GeV)$^{" + str(mag) +" }$" + 
               "(cm$^{2}$ s sr GeV)$^{-1}$")
    axes[0,0].legend(loc='upper right')
    #plt.tight_layout()

    
    if save_pdf: fig.savefig(os.path.join(desktop, pref + 'flux.pdf'))

In [ ]:
#get path of the home directory + Desktop
save_pdf = False
desktop = os.path.join(os.path.expanduser("~"),'Desktop')

kw={}
for pref, lab in [('mu_',r'\mu'), ('numu_',r'\nu_\mu'), ('nue_',r'\nu_e')]:
    kw = dict(hspace=0.1, height_ratios=[3, 1])
    fig, axes = plt.subplots(2,1,sharex=True,figsize=(7.,7.), gridspec_kw=kw,squeeze=False)
    p0 = axes[0,0].loglog(e_grid, flux[pref + 'total'],color='k', ls='-', lw=1.5)
    c0 = p0[0].get_color()
    axes[0,0].loglog(e_grid, flux[pref + 'conv'], color=c0, ls='-.', lw=1.5,
               label=r'conventional ${0}$'.format(lab))
    axes[0,0].loglog(e_grid, flux[pref + 'pr'], color=c0,ls='--', lw=1.5, 
               label='prompt ${0}$'.format(lab))
    '''
    #plot na49 results in the same plot
    p1 = axes[0,0].loglog(e_grid, flux_na49[pref + 'total'],color='orangered', ls='-', lw=2.)
    c1 = p1[0].get_color()
    axes[0,0].loglog(e_grid, flux_na49[pref + 'conv'], color=c1, ls='-.', lw=2.,
               label=r'NA49 conventional ${0}$'.format(lab))
    axes[0,0].loglog(e_grid, flux_na49[pref + 'pr'], color=c1,ls='--', lw=2., 
               label='NA49 prompt ${0}$'.format(lab))
    
    #plot EPOS results in the same plot
    p2 = axes[0,0].loglog(e_grid, flux_epos[pref + 'total'],color='limegreen', ls='-', lw=1.5)
    c2 = p2[0].get_color()
    axes[0,0].loglog(e_grid, flux_epos[pref + 'conv'], color=c2, ls='-.', lw=1.5,
               label=r'EPOS conventional ${0}$'.format(lab))
    axes[0,0].loglog(e_grid, flux_epos[pref + 'pr'], color=c2,ls='--', lw=1.5, 
               label='EPOS prompt ${0}$'.format(lab))
    '''
    axes[0,0].set_xlim(1,1e5)
    axes[0,0].set_ylim(1e-6,1)
    #axes[0,0].set_xlabel(r"$E_{{{0}}}$ [GeV]".format(lab))
    axes[0,0].set_ylabel(r"$\Phi_{" + lab + "}$ (E/GeV)$^{" + str(mag) +" }$" + 
               "(cm$^{2}$ s sr GeV)$^{-1}$")
    axes[0,0].legend(loc='upper right')
    #plt.tight_layout()
    
    #plot the ratio total/na49 for epos and sibyll
    
    #axes[1,0].plot(e_grid, flux[pref + 'total']/flux_na49[pref + 'total'],color=c0, ls='-', lw=1.5, label='Sibyll')
    #axes[1,0].plot(e_grid, flux_epos[pref + 'total']/flux_na49[pref + 'total'],color=c2, ls='-', lw=1.5,label='EPOS')
    axes[1,0].set_xlim(1,1e9)
    axes[1,0].set_ylim(0.,2.)
    axes[1,0].set_xlabel(r"$E_{{{0}}}$ [GeV]".format(lab))
    axes[1,0].set_ylabel(r"Ratio: Model / NA49")
    axes[1,0].set_xscale('log')
    axes[1,0].legend(loc='best')
    
    if save_pdf: fig.savefig(os.path.join(desktop, pref + 'flux.pdf'))

pref = 'nutau'
if np.any(flux[pref + '_pr']): #for models without prompt -> no nutau
    #plt.figure(figsize=(7., 5.5))
    kw = dict(hspace=0.1, height_ratios=[3, 1])
    fig, axes = plt.subplots(2,1,sharex=True,figsize=(7.,7.), gridspec_kw=kw,squeeze=False)
    lab = r'\nu_\tau'
    axes[0,0].loglog(e_grid, flux[pref + '_pr'], color='k', ls='-', lw=1.5,
               label='prompt ${0}$'.format(lab))
    #axes[0,0].loglog(e_grid, flux_na49[pref + '_pr'], color='k', ls='--', lw=2.,
    #           label='NA49 prompt ${0}$'.format(lab))
    #axes[0,0].loglog(e_grid, flux_epos[pref + '_pr'], color='k', ls='--', lw=1.5,
    #           label='EPOS prompt ${0}$'.format(lab))
    
    axes[0,0].set_xlim(1,1e9)
    axes[0,0].set_ylim(1e-8,1e-3)
    axes[0,0].set_xlabel(r"$E_{{{0}}}$ [GeV]".format(lab))
    axes[0,0].set_ylabel(r"$\Phi_{" + lab + "}$ (E/GeV)$^{" + str(mag) +" }$" + 
               "(cm$^{2}$ s sr GeV)$^{-1}$")
    axes[0,0].legend(loc='upper right')
    #plt.tight_layout()
    
    #axes[1,0].plot(e_grid, flux[pref + '_pr']/flux_na49[pref + '_pr'],color=c0, ls='-', lw=1.5, label='Sibyll')
    #axes[1,0].plot(e_grid, flux_epos[pref + '_pr']/flux_na49[pref + '_pr'],color=c2, ls='-', lw=1.5,label='EPOS')
    axes[1,0].set_xlim(1,1e9)
    axes[1,0].set_ylim(0.,2.)
    axes[1,0].set_xlabel(r"$E_{{{0}}}$ [GeV]".format(lab))
    axes[1,0].set_ylabel(r"Ratio: Model / NA49")
    axes[1,0].set_xscale('log')
    axes[1,0].legend(loc='best')
    
    if save_pdf: fig.savefig(os.path.join(desktop, pref + 'flux.pdf'))

Plot flavor ratios 
------------------

In [ ]:
#Plot Ratios
save_pdf = True

#Power of energy to scale the flux
mag = 3

#obtain energy grid (nver changes) of the solution for the x-axis of the plots
e_grid = mceq_run.e_grid

# total means conventional + prompt
flux_na49 = {}
flux_na49['total_mu+'] = mceq_run_na49.get_solution('total_mu+', mag)
flux_na49['total_mu-'] = mceq_run_na49.get_solution('total_mu-', mag)

flux_na49['total_numu'] = mceq_run_na49.get_solution('total_numu', mag)
flux_na49['total_antinumu'] = mceq_run_na49.get_solution('total_antinumu', mag) 
flux_na49['total_nue'] = mceq_run_na49.get_solution('total_nue', mag)
flux_na49['total_antinue'] = mceq_run_na49.get_solution('total_antinue', mag) 

flux_na49['numu_total'] = (mceq_run_na49.get_solution('total_numu', mag)
                      + mceq_run_na49.get_solution('total_antinumu', mag))

flux_na49['nue_total'] = (mceq_run_na49.get_solution('total_nue', mag)
                     + mceq_run_na49.get_solution('total_antinue', mag))



flux = {}
flux['total_mu+'] = mceq_run.get_solution('total_mu+', mag)
flux['total_mu-'] = mceq_run.get_solution('total_mu-', mag) 

flux['total_numu'] = mceq_run.get_solution('total_numu', mag)
flux['total_antinumu'] = mceq_run.get_solution('total_antinumu', mag)
flux['total_nue'] = mceq_run.get_solution('total_nue', mag)
flux['total_antinue'] = mceq_run.get_solution('total_antinue', mag) 

flux['numu_total'] = (mceq_run.get_solution('total_numu', mag)
                      + mceq_run.get_solution('total_antinumu', mag))

flux['nue_total'] = (mceq_run.get_solution('total_nue', mag)
                     + mceq_run.get_solution('total_antinue', mag))


#plot the muon charge ratio
kw = dict(hspace=0.1, height_ratios=[3, 1])
fig, axes = plt.subplots(2,1,sharex=True,figsize=(7.,5.5), gridspec_kw=kw,squeeze=False)
fig.suptitle('Muon charge ratio',fontsize=20)

for f,lab in [(flux,'Sibyll'),(flux_na49,'NA49')]:
    ratio = f['total_mu+'] / f['total_mu-']
    axes[0,0].plot(e_grid, ratio, ls='-', lw=2.,
            label=lab)

axes[1,0].plot(e_grid, ratio / (flux['total_mu+'] / flux['total_mu-']), lw=2, ls='--') 
axes[1,0].set_ylim([0.8,1.2])
axes[1,0].set_ylabel(r'$\frac{NA49}{Sibyll}$',fontsize=12)

axes[0,0].set_xlim(1,1e5)
axes[0,0].set_ylim(1.1,1.9)
#axes[0,0].set_xlim(10,1e9)
#axes[0,0].set_ylim([0.8,1.6])
axes[0,0].set_xscale('log')
axes[1,0].set_xlabel(r"$E_{\mu}$ [GeV]",fontsize=12)
axes[0,0].set_ylabel(r'$\phi(\mu^+) / \phi(\mu^-)$',fontsize=12)

axes[0,0].legend(loc='best',fontsize=12)

if save_pdf: fig.savefig(os.path.join(desktop, 'MuonChargeRatio.pdf'))


#plot the numu/antinumu and nue/antinue ratio

fig, axes = plt.subplots(2,2,sharex=True,figsize=(14.,5.5), gridspec_kw=kw,squeeze=False)
axes[0,0].set_title(r'$\nu_{\mu}/\bar{\nu}_{\mu}$',fontsize=20)
axes[0,1].set_title(r'$\nu_{e}/\bar{\nu}_{e}$',fontsize=20)

for f,lab in [(flux,'Sibyll'),(flux_na49,'NA49')]:
    ratio_mu = f['total_numu'] / f['total_antinumu']
    axes[0,0].plot(e_grid, ratio_mu, ls='-', lw=2.,
            label=lab)
    
    ratio_e = f['total_nue'] / f['total_antinue']
    axes[0,1].plot(e_grid, ratio_e, ls='-', lw=2.,
            label=lab)

axes[1,0].plot(e_grid, ratio_mu / (flux['total_numu'] / flux['total_antinumu']), lw=2, ls='--') 
axes[1,1].plot(e_grid, ratio_e / (flux['total_nue'] / flux['total_antinue']), lw=2, ls='--')

axes[1,0].set_ylim([0.8,1.2])
axes[1,1].set_ylim([0.8,1.2])
axes[1,0].set_ylabel(r'$\frac{NA49}{Sibyll}$',fontsize=12)

axes[0,0].set_xlim(10,1e9)
axes[0,0].set_ylim([0.8,2.5])
axes[0,1].set_ylim([0.8,2.5])
axes[0,0].set_xscale('log')
axes[1,1].set_xlabel(r"$E$ [GeV]",fontsize=12)
axes[1,0].set_xlabel(r"$E$ [GeV]",fontsize=12)
axes[0,0].set_ylabel(r'$\phi(\nu_{\mu}) / \phi(\bar{\nu}_{\mu})$',fontsize=12)
axes[0,1].set_ylabel(r'$\phi(\nu_{e}) / \phi(\bar{\nu}_{e})$',fontsize=12)

axes[0,0].legend(loc='best',fontsize=12)
if save_pdf: fig.savefig(os.path.join(desktop, 'Neutrino_Antineutrino_Ratio.pdf'))


#plot the numu/nue ratio

fig, axes = plt.subplots(2,1,sharex=True,figsize=(7.,5.5), gridspec_kw=kw,squeeze=False)
axes[0,0].set_title(r'$(\nu_{\mu}+\bar{\nu}_{\mu})/(\nu_{e}+\bar{\nu}_{e})$',fontsize=20)

for f,lab in [(flux,'Sibyll'),(flux_na49,'NA49')]:
    ratio = f['numu_total'] / f['nue_total']
    axes[0,0].plot(e_grid, ratio, ls='-', lw=2.,
            label=lab)
    
    
axes[1,0].plot(e_grid, ratio / (flux['numu_total'] / flux['nue_total']), lw=2, ls='--') 


axes[1,0].set_ylim([0.8,1.2])
axes[1,0].set_ylabel(r'$\frac{NA49}{Sibyll}$',fontsize=12)

axes[0,0].set_xlim(10,1e9)
axes[0,0].set_ylim([0.8,30.])
axes[0,0].set_xscale('log')
axes[1,0].set_xlabel(r"$E$ [GeV]",fontsize=12)
axes[0,0].set_ylabel(r'$\phi(\nu_{\mu}) / \phi(\nu_{e})$',fontsize=12)


axes[0,0].legend(loc='best',fontsize=12)
if save_pdf: fig.savefig(os.path.join(desktop, 'NeutrinoFlavorRatio.pdf'))


Save as in ASCII file for other types of processing
---------------------------------------------------

In [ ]:
np.savetxt(open(os.path.join(desktop, 'my_flux_calculation.txt'),'w'),
zip(e_grid, 
    flux['mu_conv'],flux['mu_pr'],flux['mu_total'],
    flux['numu_conv'],flux['numu_pr'],flux['numu_total'],
    flux['nue_conv'],flux['nue_pr'],flux['nue_total'],
    flux['nutau_pr']),
fmt='%6.5E',
header=('lepton flux in scaled with E**{0}. Order (E, mu_conv, mu_pr, mu_total, ' +
        'numu_conv, numu_pr, numu_total, nue_conv, nue_pr, nue_total, ' +
        'nutau_pr').format(mag)
)